In [29]:
#Read the file 

In [30]:
with open('Data/input.txt', 'r') as f:

    text = f.read()

In [31]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [32]:
#process the text , rmove url and some charcters

In [33]:
import re

def preprocess_text(text):
    
    if not isinstance(text, str):
        return ""

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    text = re.sub(r"/[^\s]+", "", text)

    text = re.sub(r'\n', '', text)

    return text.strip()

In [34]:
text = preprocess_text(text)

In [35]:
text[:100]

'First Citizen:Before we proceed any further, hear me speak.All:Speak, speak.First Citizen:You are al'

In [36]:
print('Lenght of my text', len(text))

Lenght of my text 1075394


In [37]:
chars = sorted(list(set(text)))

vocab = len(chars)

print(''.join(chars))
print(vocab)

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
64


In [38]:
# Tokeization in the character level

In [39]:
ztoa = {i:k for i,k in enumerate(chars)}
atoz = {k:i for i,k in enumerate(chars)}

encoding = lambda word: [atoz[w] for w in word]
decoding = lambda word: "".join([ztoa[w] for w in word])

print(encoding('holla hye'))
print(decoding(encoding('holla hye')))

[45, 52, 49, 49, 38, 0, 45, 62, 42]
holla hye


In [40]:
import torch
torch.manual_seed(1337)

encoding_text = torch.tensor(encoding(text), dtype= torch.long)

print(encoding_text.shape, encoding_text.dtype)
print(encoding_text[: 100])

torch.Size([1075394]) torch.int64
tensor([17, 46, 55, 56, 57,  0, 14, 46, 57, 46, 63, 42, 51,  9, 13, 42, 43, 52,
        55, 42,  0, 60, 42,  0, 53, 55, 52, 40, 42, 42, 41,  0, 38, 51, 62,  0,
        43, 58, 55, 57, 45, 42, 55,  5,  0, 45, 42, 38, 55,  0, 50, 42,  0, 56,
        53, 42, 38, 48,  7, 12, 49, 49,  9, 30, 53, 42, 38, 48,  5,  0, 56, 53,
        42, 38, 48,  7, 17, 46, 55, 56, 57,  0, 14, 46, 57, 46, 63, 42, 51,  9,
        36, 52, 58,  0, 38, 55, 42,  0, 38, 49])


In [41]:
#split train validation

In [42]:
n = int(0.9*len(encoding_text))

train_data = encoding_text[:n]
valid_data = encoding_text[n:]

In [43]:
# Training with all the data at once is costly, but we can train it with chuncks

In [44]:
train_data[: 8]

tensor([17, 46, 55, 56, 57,  0, 14, 46])

In [45]:
train_data[1: 9]

tensor([46, 55, 56, 57,  0, 14, 46, 57])

In [46]:
block_size = 8
torch.manual_seed(1337)
x = train_data[: block_size]

y = train_data[1: block_size + 1]

for i in range(len(train_data[: block_size])):

    o = x[:i+1]
    j = y[i]

    print(f'whne input is {o} target is {j}')

whne input is tensor([17]) target is 46
whne input is tensor([17, 46]) target is 55
whne input is tensor([17, 46, 55]) target is 56
whne input is tensor([17, 46, 55, 56]) target is 57
whne input is tensor([17, 46, 55, 56, 57]) target is 0
whne input is tensor([17, 46, 55, 56, 57,  0]) target is 14
whne input is tensor([17, 46, 55, 56, 57,  0, 14]) target is 46
whne input is tensor([17, 46, 55, 56, 57,  0, 14, 46]) target is 57


In [47]:
# Random sampling

In [48]:
batch_size = 4

def random_sampling(split):

    data = train_data if split == 'train' else valid_data

    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

x, y = random_sampling('train')

print(x.shape, y.shape)
print('\n', x)
print('\n', y)
print('\n')
for i in range(4):

    for j in range(8):

        x_l = x[i][:j+1]
        y_l = y[i][j]

        print(f"when input is {x_l} the target is {y_l}")

torch.Size([4, 8]) torch.Size([4, 8])

 tensor([[44, 42, 57,  5, 31, 45, 42, 51],
        [ 0, 49, 52, 51, 44, 42, 55,  0],
        [57,  9, 36, 52, 58, 55,  0, 43],
        [ 0, 45, 42,  0, 46, 56,  0, 40]])

 tensor([[42, 57,  5, 31, 45, 42, 51,  0],
        [49, 52, 51, 44, 42, 55,  0, 56],
        [ 9, 36, 52, 58, 55,  0, 43, 46],
        [45, 42,  0, 46, 56,  0, 40, 52]])


when input is tensor([44]) the target is 42
when input is tensor([44, 42]) the target is 57
when input is tensor([44, 42, 57]) the target is 5
when input is tensor([44, 42, 57,  5]) the target is 31
when input is tensor([44, 42, 57,  5, 31]) the target is 45
when input is tensor([44, 42, 57,  5, 31, 45]) the target is 42
when input is tensor([44, 42, 57,  5, 31, 45, 42]) the target is 51
when input is tensor([44, 42, 57,  5, 31, 45, 42, 51]) the target is 0
when input is tensor([0]) the target is 49
when input is tensor([ 0, 49]) the target is 52
when input is tensor([ 0, 49, 52]) the target is 51
when input is

In [49]:
x

tensor([[44, 42, 57,  5, 31, 45, 42, 51],
        [ 0, 49, 52, 51, 44, 42, 55,  0],
        [57,  9, 36, 52, 58, 55,  0, 43],
        [ 0, 45, 42,  0, 46, 56,  0, 40]])

In [61]:
import torch 
import torch.nn as nn
from torch.nn import functional as functional
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        self.embeding = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, target):

        logits = self.embeding(idx)
    
        return logits

m = BigramLanguageModel(vocab_size= vocab)
logits = m(x, y)

print(logits.shape)
print('\n', logits)

torch.Size([4, 8, 64])

 tensor([[[-0.4198, -0.6435, -0.3311,  ..., -0.9595,  1.8561,  0.1731],
         [-1.0184,  2.2253,  0.5137,  ..., -0.4551, -0.0516, -0.4424],
         [ 1.0237, -0.1184,  0.1446,  ...,  0.7763, -0.8460,  0.8437],
         ...,
         [-0.8251,  1.4378, -1.6157,  ..., -0.6972,  0.1416,  1.1991],
         [-1.0184,  2.2253,  0.5137,  ..., -0.4551, -0.0516, -0.4424],
         [-0.5655,  0.1863,  0.4466,  ...,  0.3164,  1.0056, -0.9158]],

        [[ 0.1808, -0.0700, -0.3596,  ...,  1.5333,  1.6097, -0.4032],
         [-1.5012, -1.7006,  1.2642,  ..., -1.6868, -0.9292, -1.2395],
         [-1.9572,  0.0552,  0.2706,  ..., -0.0134, -1.9390,  0.6582],
         ...,
         [-1.0184,  2.2253,  0.5137,  ..., -0.4551, -0.0516, -0.4424],
         [-1.3360, -0.9757, -0.5134,  ...,  1.8886,  2.6879,  0.9940],
         [ 0.1808, -0.0700, -0.3596,  ...,  1.5333,  1.6097, -0.4032]],

        [[ 1.0237, -0.1184,  0.1446,  ...,  0.7763, -0.8460,  0.8437],
         [-1.0277,  